Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  
  del save  # hint to help gc free up memory
  
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [7]:
batch_size = 128
beta = 1e-5

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  beta_regul = tf.placeholder(tf.float32)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Regularization
  regularizer = tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
  loss = loss + beta * regularizer

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [10]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Graph initialized")
  
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    dic = {tf_train_dataset: batch_data, 
           tf_train_labels: batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=dic)
    
    if (step % 100 == 0):
      print("\nMinibatch loss at step %d: %.3f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
      
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Graph initialized

Minibatch loss at step 0: 17.422
Minibatch accuracy: 4.7%
Validation accuracy: 6.9%

Minibatch loss at step 100: 2.570
Minibatch accuracy: 74.2%
Validation accuracy: 71.4%

Minibatch loss at step 200: 2.464
Minibatch accuracy: 68.8%
Validation accuracy: 74.3%

Minibatch loss at step 300: 2.695
Minibatch accuracy: 70.3%
Validation accuracy: 74.6%

Minibatch loss at step 400: 2.366
Minibatch accuracy: 71.1%
Validation accuracy: 75.2%

Minibatch loss at step 500: 1.695
Minibatch accuracy: 78.1%
Validation accuracy: 75.1%

Minibatch loss at step 600: 1.620
Minibatch accuracy: 78.1%
Validation accuracy: 76.0%

Minibatch loss at step 700: 1.145
Minibatch accuracy: 82.8%
Validation accuracy: 76.1%

Minibatch loss at step 800: 1.302
Minibatch accuracy: 78.1%
Validation accuracy: 76.5%

Minibatch loss at step 900: 1.372
Minibatch accuracy: 73.4%
Validation accuracy: 76.4%

Minibatch loss at step 1000: 1.568
Minibatch accuracy: 76.6%
Validation accuracy: 76.3%
Test accuracy: 8

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [21]:
batch_size = 128
beta = 0.005

# Extracting a subset of data for training
train_subset = train_dataset[:1000, :]
train_sublabels = train_labels[:1000]

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  beta_regul = tf.placeholder(tf.float32)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Regularization
  regularizer = tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
  loss = loss + beta * regularizer

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [27]:
num_steps = 2001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Graph initialized")
  
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_sublabels.shape[0] - batch_size)
    
    # Generate a minibatch.
    batch_data = train_subset[offset:(offset + batch_size), :]
    batch_labels = train_sublabels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    dic = {tf_train_dataset : batch_data, 
           tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=dic)
    
    if (step % 100 == 0):
      print("\nStep %d" % step)
      print("Minibatch loss: %.3f" % l)
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
      
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Graph initialized

Step 0
Minibatch loss: 14.478
Minibatch accuracy: 11.7%
Validation accuracy: 15.8%

Step 100
Minibatch loss: 1.997
Minibatch accuracy: 73.4%
Validation accuracy: 64.6%

Step 200
Minibatch loss: 0.448
Minibatch accuracy: 93.8%
Validation accuracy: 66.2%

Step 300
Minibatch loss: 0.722
Minibatch accuracy: 88.3%
Validation accuracy: 66.8%

Step 400
Minibatch loss: 0.311
Minibatch accuracy: 93.8%
Validation accuracy: 67.0%

Step 500
Minibatch loss: 0.115
Minibatch accuracy: 99.2%
Validation accuracy: 67.5%

Step 600
Minibatch loss: 0.247
Minibatch accuracy: 97.7%
Validation accuracy: 67.8%

Step 700
Minibatch loss: 0.118
Minibatch accuracy: 99.2%
Validation accuracy: 67.9%

Step 800
Minibatch loss: 0.086
Minibatch accuracy: 99.2%
Validation accuracy: 68.2%

Step 900
Minibatch loss: 0.158
Minibatch accuracy: 99.2%
Validation accuracy: 68.3%

Step 1000
Minibatch loss: 0.081
Minibatch accuracy: 99.2%
Validation accuracy: 68.5%

Step 1100
Minibatch loss: 0.067
Minibatch accu

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [100]:
num_nodes= 1024
batch_size = 128
beta = 0.005

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
    biases_1 = tf.Variable(tf.zeros([num_nodes]))
    weights_2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    hidden_layer= tf.nn.relu(logits_1)
    
    # Dropout on hidden layer: RELU layer
    keep_prob = tf.placeholder("float")
    hidden_layer_dropout = tf.nn.dropout(hidden_layer, keep_prob)
    
    logits_2 = tf.matmul(hidden_layer_dropout, weights_2) + biases_2
    
    # Normal loss function
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits_2, tf_train_labels))
    
    
    # Loss function with L2 Regularization with beta=0.005
    regularizers = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)
    loss = tf.reduce_mean(loss + beta * regularizers)

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training
    train_prediction = tf.nn.softmax(logits_2)
    
    # Predictions for validation 
    logits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    hidden_layer = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(hidden_layer, weights_2) + biases_2
    
    valid_prediction = tf.nn.softmax(logits_2)
    
    # Predictions for test
    logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    hidden_layer = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(hidden_layer, weights_2) + biases_2
    
    test_prediction =  tf.nn.softmax(logits_2)

In [101]:
num_steps = 2001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Graph initialized")
    
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        dic = {tf_train_dataset: batch_data, 
               tf_train_labels: batch_labels, 
               keep_prob: 0.5}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict = dic)
        
        if (step % 200 == 0):
            print("\nStep {}".format(step))
            print("Minibatch loss: {:.2f}".format(l))
            print("Minibatch accuracy: {:.2f}%".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {:.2f}%".format(accuracy(valid_prediction.eval(), valid_labels)))
            
    print("\nTest accuracy: {:.2f}%".format(accuracy(test_prediction.eval(), test_labels)))

Graph initialized

Step 0
Minibatch loss: 1998.23
Minibatch accuracy: 8.59%
Validation accuracy: 38.19%

Step 200
Minibatch loss: 593.93
Minibatch accuracy: 67.97%
Validation accuracy: 78.07%

Step 400
Minibatch loss: 211.54
Minibatch accuracy: 74.22%
Validation accuracy: 81.40%

Step 600
Minibatch loss: 77.54
Minibatch accuracy: 78.91%
Validation accuracy: 83.06%

Step 800
Minibatch loss: 28.67
Minibatch accuracy: 83.59%
Validation accuracy: 84.52%

Step 1000
Minibatch loss: 10.97
Minibatch accuracy: 80.47%
Validation accuracy: 84.69%

Step 1200
Minibatch loss: 4.33
Minibatch accuracy: 88.28%
Validation accuracy: 84.47%

Step 1400
Minibatch loss: 2.15
Minibatch accuracy: 85.16%
Validation accuracy: 84.88%

Step 1600
Minibatch loss: 1.27
Minibatch accuracy: 83.59%
Validation accuracy: 84.83%

Step 1800
Minibatch loss: 0.88
Minibatch accuracy: 87.50%
Validation accuracy: 84.58%

Step 2000
Minibatch loss: 0.94
Minibatch accuracy: 78.91%
Validation accuracy: 85.03%
Test accuracy: 90.97%


Extreme Overfitting case as follows:

In [46]:
batch_size = 128
beta = 0.005

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
    biases_1 = tf.Variable(tf.zeros([num_nodes]))
    weights_2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    hidden_layer= tf.nn.relu(logits_1)
    
    # Dropout on hidden layer: RELU layer
    keep_prob = tf.placeholder("float")
    hidden_layer_dropout = tf.nn.dropout(hidden_layer, keep_prob)
    
    logits_2 = tf.matmul(hidden_layer_dropout, weights_2) + biases_2
    
    # Normal loss function
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits_2, tf_train_labels))
    
    
    # Loss function with L2 Regularization with beta=0.005
    regularizers = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)
    loss = tf.reduce_mean(loss + beta * regularizers)

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training
    train_prediction = tf.nn.softmax(logits_2)
    
    # Predictions for validation 
    logits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    hidden_layer = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(hidden_layer, weights_2) + biases_2
    
    valid_prediction = tf.nn.softmax(logits_2)
    
    # Predictions for test
    logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    hidden_layer = tf.nn.relu(logits_1)
    logits_2 = tf.matmul(hidden_layer, weights_2) + biases_2
    
    test_prediction =  tf.nn.softmax(logits_2)

In [52]:
num_steps = 2001

# Extracting a subset of data for training
train_subset = train_dataset[:1000, :]
train_sublabels = train_labels[:1000]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Graph initialized")
  
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_sublabels.shape[0] - batch_size)
    
    # Generate a minibatch.
    batch_data = train_subset[offset:(offset + batch_size), :]
    batch_labels = train_sublabels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    
    dic = {tf_train_dataset: batch_data, 
           tf_train_labels: batch_labels,  
           keep_prob: 0.5}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict= dic)
    
    if (step % 200 == 0):
      print("\nStep %d" % step)
      print("Minibatch loss: %.2f" % l)
      print("Minibatch accuracy: %.2f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.2f%%" % accuracy(valid_prediction.eval(), valid_labels))
      
  print("Test accuracy: %.2f%%" % accuracy(test_prediction.eval(), test_labels))

Graph initialized

Step 0
Minibatch loss: 2066.72
Minibatch accuracy: 10.94%
Validation accuracy: 29.86%

Step 200
Minibatch loss: 581.40
Minibatch accuracy: 100.00%
Validation accuracy: 77.55%

Step 400
Minibatch loss: 213.89
Minibatch accuracy: 100.00%
Validation accuracy: 78.16%

Step 600
Minibatch loss: 78.62
Minibatch accuracy: 100.00%
Validation accuracy: 78.29%

Step 800
Minibatch loss: 28.94
Minibatch accuracy: 99.22%
Validation accuracy: 78.96%

Step 1000
Minibatch loss: 10.75
Minibatch accuracy: 100.00%
Validation accuracy: 79.79%

Step 1200
Minibatch loss: 4.13
Minibatch accuracy: 100.00%
Validation accuracy: 79.80%

Step 1400
Minibatch loss: 1.71
Minibatch accuracy: 99.22%
Validation accuracy: 79.96%

Step 1600
Minibatch loss: 0.82
Minibatch accuracy: 100.00%
Validation accuracy: 80.05%

Step 1800
Minibatch loss: 0.50
Minibatch accuracy: 100.00%
Validation accuracy: 80.03%

Step 2000
Minibatch loss: 0.37
Minibatch accuracy: 100.00%
Validation accuracy: 79.85%
Test accuracy:

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [127]:
import math

batch_size = 128
beta = 0.005
dropout = True

num_in = image_size * image_size
num_hidden1 = 1024
num_hidden2 = int(num_hidden1*0.5)
num_hidden3 = int(num_hidden1*0.25)

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, num_in))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights_1 = tf.Variable(tf.truncated_normal([num_in, num_hidden1], stddev=math.sqrt(2.0/num_in)))
    biases_1 = tf.Variable(tf.zeros([num_hidden1]))
    
    weights_2 = tf.Variable(tf.truncated_normal([num_hidden1, num_hidden2], stddev=math.sqrt(2.0/num_hidden1)))
    biases_2 = tf.Variable(tf.zeros([num_hidden2]))
    
    weights_3 = tf.Variable(tf.truncated_normal([num_hidden2, num_hidden3], stddev=math.sqrt(2.0/num_hidden2)))
    biases_3 = tf.Variable(tf.zeros([num_hidden3]))
    
    weights_out = tf.Variable(tf.truncated_normal([num_hidden3, num_labels], stddev=math.sqrt(2.0/num_hidden3)))
    biases_out = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    hidden_layer1 = tf.nn.relu(logits_1)
    if dropout:
        hidden_layer1 = tf.nn.dropout(hidden_layer1, 0.5)
    
    logits_2 = tf.matmul(hidden_layer1, weights_2) + biases_2
    hidden_layer2 = tf.nn.relu(logits_2)
    if dropout:
        hidden_layer2 = tf.nn.dropout(hidden_layer2, 0.5)
        
    logits_3 = tf.matmul(hidden_layer2, weights_3) + biases_3
    hidden_layer3 = tf.nn.relu(logits_3)
    if dropout:
        hidden_layer3 = tf.nn.dropout(hidden_layer3, 0.5)
        
    logits_out = tf.matmul(hidden_layer3, weights_out) + biases_out

    # Normal loss function
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits_out, tf_train_labels))
    
    
    # Loss function with L2 Regularization with beta=0.005
    regularizers = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + \
                    tf.nn.l2_loss(weights_3) + tf.nn.l2_loss(weights_out)
    loss = tf.reduce_mean(loss + beta * regularizers)

    # Optimizer.
    gstep = tf.Variable(0)  # count the number of steps taken.
    lrate = tf.train.exponential_decay(0.5, gstep,1e5, 0.96)
    optimizer = tf.train.GradientDescentOptimizer(lrate).minimize(loss, global_step=gstep)
 
    # Predictions for the training
    train_pred = tf.nn.softmax(logits_out)
    
    # Predictions for validation 
    logits_valid_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    valid_layer1 = tf.nn.relu(logits_valid_1)
    
    logits_valid_2 = tf.matmul(valid_layer1, weights_2) + biases_2
    valid_layer2 = tf.nn.relu(logits_valid_2)
        
    logits_valid_3 = tf.matmul(valid_layer2, weights_3) + biases_3
    valid_layer3 = tf.nn.relu(logits_valid_3)
        
    logits_valid_out = tf.matmul(valid_layer3, weights_out) + biases_out
    valid_pred = tf.nn.softmax(logits_valid_out)
    
    # Predictions for test
    logits_test_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    test_layer1 = tf.nn.relu(logits_test_1)
    
    logits_test_2 = tf.matmul(test_layer1, weights_2) + biases_2
    test_layer2 = tf.nn.relu(logits_test_2)
        
    logits_test_3 = tf.matmul(test_layer2, weights_3) + biases_3
    test_layer3 = tf.nn.relu(logits_test_3)
        
    logits_test_out = tf.matmul(test_layer3, weights_out) + biases_out
    test_pred = tf.nn.softmax(logits_test_out)

In [128]:
num_steps = 10001

# Extracting a subset of data for training

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Graph initialized")
  
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    
    dic = {tf_train_dataset: batch_data, 
           tf_train_labels: batch_labels}
    _, l, batch_pred = session.run([optimizer, loss, train_pred], feed_dict= dic)
    
    if (step % 400 == 0):
      print("\nStep %d" % step)
      print("Minibatch loss: %.2f" % l)
      print("Minibatch accuracy: %.2f%%" % accuracy(batch_pred, batch_labels))
      print("Validation accuracy: %.2f%%" % accuracy(valid_pred.eval(), valid_labels))
      
  print("\nTest accuracy: %.2f%%" % accuracy(test_pred.eval(), test_labels))

Graph initialized

Step 0
Minibatch loss: 9.77
Minibatch accuracy: 8.59%
Validation accuracy: 20.41%

Step 400
Minibatch loss: 1.86
Minibatch accuracy: 80.47%
Validation accuracy: 82.52%

Step 800
Minibatch loss: 0.98
Minibatch accuracy: 81.25%
Validation accuracy: 79.30%

Step 1200
Minibatch loss: 0.64
Minibatch accuracy: 88.28%
Validation accuracy: 83.68%

Step 1600
Minibatch loss: 0.86
Minibatch accuracy: 80.47%
Validation accuracy: 84.10%

Step 2000
Minibatch loss: 0.99
Minibatch accuracy: 76.56%
Validation accuracy: 85.19%

Step 2400
Minibatch loss: 0.93
Minibatch accuracy: 80.47%
Validation accuracy: 83.99%

Step 2800
Minibatch loss: 0.81
Minibatch accuracy: 83.59%
Validation accuracy: 84.72%

Step 3200
Minibatch loss: 0.89
Minibatch accuracy: 80.47%
Validation accuracy: 84.57%

Step 3600
Minibatch loss: 0.70
Minibatch accuracy: 90.62%
Validation accuracy: 84.24%

Step 4000
Minibatch loss: 0.85
Minibatch accuracy: 80.47%
Validation accuracy: 84.11%

Step 4400
Minibatch loss: 0.83